# Welcome to RAG week!! (OpenRouter Version)

## Expert Knowledge Worker

### A question answering Assistant that is an expert knowledge worker
### To be used by employees of Insurellm, an Insurance Tech company
### The AI assistant needs to be accurate and the solution should be low cost.

This project will use RAG (Retrieval Augmented Generation) to ensure our question/answering assistant has high accuracy.

This first implementation will use a simplistic, brute-force type of RAG..

**Modified to work with OpenRouter API**

In [43]:
import os
import glob
from dotenv import load_dotenv
from pathlib import Path
import gradio as gr
from openai import OpenAI

In [44]:
# Setting up OpenRouter
load_dotenv(override=True)

# OpenRouter configuration
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')
if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:8]}")
else:
    print("OpenRouter API Key not set")

MODEL = "deepseek/deepseek-chat"

# Initialize OpenAI client with OpenRouter endpoint
openai = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=openrouter_api_key,
)

OpenRouter API Key exists and begins sk-or-v1


In [45]:
print(f"Using model: {MODEL}")

Using model: deepseek/deepseek-chat


### Let's read in all employee data into a dictionary

In [46]:
knowledge = {}

filenames = glob.glob("knowledge-base/employees/*")

for filename in filenames:
    name = Path(filename).stem.split(' ')[-1]
    print(name, filename)
    with open(filename, "r", encoding="utf-8") as f:
        knowledge[name.lower()] = f.read()

Kim knowledge-base/employees/David Kim.md
Patel knowledge-base/employees/Nina Patel.md
Sharma knowledge-base/employees/Priya Sharma.md
Chen knowledge-base/employees/Alex Chen.md
Chen knowledge-base/employees/Robert Chen.md
Spencer knowledge-base/employees/Oliver Spencer.md
Foster knowledge-base/employees/Amanda Foster.md
Tran knowledge-base/employees/Emily Tran.md
Blake knowledge-base/employees/Jordan Blake.md
Zhang knowledge-base/employees/Kevin Zhang.md
Thompson knowledge-base/employees/Maya Thompson.md
Adams knowledge-base/employees/Jennifer Adams.md
Liu knowledge-base/employees/Jessica Liu.md
Lancaster knowledge-base/employees/Avery Lancaster.md
Thompson knowledge-base/employees/Maxine Thompson.md
Park knowledge-base/employees/Daniel Park.md
Greene knowledge-base/employees/Samantha Greene.md
Johnson knowledge-base/employees/Marcus Johnson.md
Thomson knowledge-base/employees/Alex Thomson.md
O'Brien knowledge-base/employees/Michael O'Brien.md
Martinez knowledge-base/employees/Rachel 

In [47]:
# Show available employees
print(f"Loaded {len(knowledge)} employee records")
print("Available employees:", list(knowledge.keys()))

Loaded 30 employee records
Available employees: ['kim', 'patel', 'sharma', 'chen', 'spencer', 'foster', 'tran', 'blake', 'zhang', 'thompson', 'adams', 'liu', 'lancaster', 'park', 'greene', 'johnson', 'thomson', "o'brien", 'martinez', 'williams', 'rivera', 'trenton', 'anderson', 'harper', 'rodriguez', 'wilson', 'bishop', 'carter', 'brooks', 'walker']


In [48]:
# Load product data
filenames = glob.glob("knowledge-base/products/*")

for filename in filenames:
    name = Path(filename).stem
    print(name, filename)
    with open(filename, "r", encoding="utf-8") as f:
        knowledge[name.lower()] = f.read()

Rellm knowledge-base/products/Rellm.md
Claimllm knowledge-base/products/Claimllm.md
Bizllm knowledge-base/products/Bizllm.md
Lifellm knowledge-base/products/Lifellm.md
Healthllm knowledge-base/products/Healthllm.md
Markellm knowledge-base/products/Markellm.md
Homellm knowledge-base/products/Homellm.md
Carllm knowledge-base/products/Carllm.md


In [49]:
print(f"Total knowledge base entries: {len(knowledge)}")
print("All available keys:", list(knowledge.keys()))

Total knowledge base entries: 38
All available keys: ['kim', 'patel', 'sharma', 'chen', 'spencer', 'foster', 'tran', 'blake', 'zhang', 'thompson', 'adams', 'liu', 'lancaster', 'park', 'greene', 'johnson', 'thomson', "o'brien", 'martinez', 'williams', 'rivera', 'trenton', 'anderson', 'harper', 'rodriguez', 'wilson', 'bishop', 'carter', 'brooks', 'walker', 'rellm', 'claimllm', 'bizllm', 'lifellm', 'healthllm', 'markellm', 'homellm', 'carllm']


In [50]:
SYSTEM_PREFIX = """
You represent Insurellm, the Insurance Tech company.
You are an expert in answering questions about Insurellm; its employees and its products.
You are provided with additional context that might be relevant to the user's question.
Give brief, accurate answers. If you don't know the answer, say so.

Relevant context:
"""

In [51]:
def get_relevant_context(message):
    text = ''.join(ch for ch in message if ch.isalpha() or ch.isspace())
    words = text.lower().split()
    return [knowledge[word] for word in words if word in knowledge]

In [52]:
def additional_context(message):
    relevant_context = get_relevant_context(message)
    if not relevant_context:
        result = "There is no additional context relevant to the user's question."
    else:
        result = "The following additional context might be relevant in answering the user's question:\n\n"
        result += "\n\n".join(relevant_context)
    return result

In [ ]:
def chat(message, history):
    system_message = SYSTEM_PREFIX + additional_context(message)
    print(system_message)
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    print(messages)

    try:
        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages,
            max_tokens=1000,
            temperature=0
        )
        print(response)
        return response.choices[0].message.content
    except Exception as e:
        print(e)
        return f"Error: {str(e)}"

## Test the chat function

In [54]:
# Test the chat function
test_response = chat("Who is Lancaster?", [])
print(test_response)

ChatCompletion(id='gen-1767898062-Q2euDWdWEgx58tH9kx0F', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Avery Lancaster is the Co-Founder and CEO of Insurellm. She has led the company since its inception in 2015, driving innovation and growth in the Insurance Tech sector. Currently based in San Francisco, she earns a $225,000 salary and has been recognized for her leadership, particularly in navigating challenges like the COVID-19 pandemic and fostering diversity initiatives. Prior to Insurellm, she held roles at Innovate Insurance Solutions and Edge Analytics.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning=None), native_finish_reason='stop')], created=1767898062, model='deepseek/deepseek-chat', object='chat.completion', service_tier=None, system_fingerprint='', usage=CompletionUsage(completion_tokens=95, prompt_tokens=989, total_tokens=1084, completion_tokens_details=C

## Launch Gradio Interface

In [55]:
# Launch Gradio chat interface
view = gr.ChatInterface(
    chat,
    type="messages",
    title="Insurellm Knowledge Assistant (OpenRouter)",
    description=f"Ask questions about Insurellm employees and products. Using {MODEL} via OpenRouter."
).launch(inbrowser=True, share=False)

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


ChatCompletion(id='gen-1767898076-9O7Obo4CYCSYNvbV6aEc', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Insurellm is an Insurance Tech company focused on leveraging technology to innovate and improve insurance solutions. If you have specific questions about its products or services, feel free to ask!', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning=None), native_finish_reason='stop')], created=1767898076, model='deepseek/deepseek-chat', object='chat.completion', service_tier=None, system_fingerprint='', usage=CompletionUsage(completion_tokens=36, prompt_tokens=90, total_tokens=126, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=None, audio_tokens=None, reasoning_tokens=0, rejected_prediction_tokens=None, image_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0, video_tokens=0), cost=8.28e-05, is_byok=False, c